In [ ]:
import os

os.environ["OPENAI_API_KEY"] = ""

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = ""
os.environ["LANGCHAIN_PROJECT"] = "Pruefer"

In [ ]:
import redis

redis_client = redis.Redis(host='localhost', port=6379, db=0, decode_responses=True)

In [ ]:
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain_openai.chat_models import ChatOpenAI
from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain_core.runnables import RunnablePassthrough

system_prompt_initial = """
Du bist Englischlehrer.
Du prüfst ob die Zeitform richtig benutzt wurde und der Satz somit grammatikalisch korrekt ist.
"""

# Get the prompt to use - you can modify this!
prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template(system_prompt_initial),
        MessagesPlaceholder(variable_name="messages"),
        (
            "system",
            """Der Schüler hatte als Aufgabe den deutschen Satz erhalten: {examiner_sentence_german}
            
                Der Schüler hätte den Satz in der Zeitform {examiner_requested_tense} übersetzen sollen.
                
                Hat der Schüler die Aufgabe richtig beantwortet?
                Antwort ausschließlich mit TRUE oder FALSE
        """,
        ),
    ]
)

llm = ChatOpenAI(
    #model="gpt-4-0125-preview",
    model="gpt-3.5-turbo-0125",
    streaming=True,
    temperature=0.0,
)

grammar_checker = prompt | llm

In [ ]:
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain_openai.chat_models import ChatOpenAI
from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain_core.runnables import RunnablePassthrough

system_prompt_initial = """Du bist Knowledge Master und koordinierst die Tools.
"""

# Get the prompt to use - you can modify this!
prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template(system_prompt_initial),
        (
            "human",
            """Vorhandene Records:
            '''
            {knowledge}
            '''
            
            Prüfe anhand der vorhandenen Einträge ob es einen inhaltlich gleichen Eintrag gibt wie diesen hier:
            '''
            {mistakes_made}
            '''

            Wenn es einen Eintrag bereits der das gleiche meint, bist du fertig.
            Wenn es noch keinen Eintrag gibt, erstelle einen solchen neuen Eintrag.
            Wenn sich das Sachverhalte inhaltlich geändert hat, modifiziere diesen Eintrag          
        """,
        ),
    ]
)

llm = ChatOpenAI(
    model="gpt-4-0125-preview",
    #model="gpt-3.5-turbo-0125",
    streaming=True,
    temperature=0.0,
)

tools = [convert_to_openai_function(t) for t in agent_tools]

examiner_runnable = prompt | llm.bind_tools(tools)

In [ ]:
import operator
from typing import TypedDict, Sequence, Annotated
from langchain_core.messages import BaseMessage


class AgentState(TypedDict):
    messages: Sequence[BaseMessage]
    #messages: Annotated[Sequence[BaseMessage], operator.add]
    bewerter_shortener_scratchpad: Annotated[Sequence[BaseMessage], operator.add]

    #Context from Examiner Agent
    student_sentence_english: str

    examiner_sentence_german: str
    examiner_sentence_english: str
    examiner_requested_tense: str
    examiner_topics: str

    #Decision Variables:
    exercise_correct: str
    grammar_correct: str

    problem: str

    mistakes_made: str
    
    knowledge: str

In [ ]:
from sprachtrainer.agents import bewerter_agent
from sprachtrainer.agents import empfehler_agent
from sprachtrainer.agents import shortener_agent
import json
from langchain_core.messages import ToolMessage
from langgraph.prebuilt import ToolInvocation


def call_examiner(state):
    mistakes_made = state["mistakes_made"]
    response = examiner_runnable.invoke(
        {"mistakes_made": mistakes_made,
         "knowledge": get_knowledge_comma_separated()}
    )
    return {"messages": [response]}


def call_grammar_checker(state):
    messages = state["messages"]
    examiner_sentence_german = state["examiner_sentence_german"]
    examiner_requested_tense = state["examiner_requested_tense"]
    response = grammar_checker.invoke(
        {"messages": messages,
         "examiner_sentence_german": examiner_sentence_german,
         "examiner_requested_tense": examiner_requested_tense
         }
    )
    return {"grammar_correct": "TRUE" in response.content and "yes" or "no"}


def call_bewerter(state):
    student_sentence_english = state["student_sentence_english"]
    examiner_sentence_english = state["examiner_sentence_english"]
    bewerter_shortener_scratchpad = state["bewerter_shortener_scratchpad"]

    response = bewerter_agent.getBewerterAgent().invoke(
        {
            "student_sentence_english": student_sentence_english,
            "examiner_sentence_english": examiner_sentence_english,
            "bewerter_shortener_scratchpad": bewerter_shortener_scratchpad  
        }
    )

    # Rückgabe der Analyseergebnisse als Dictionary
    return {"mistakes_made": response.content}


def call_shortener(state):
    mistakes_made = state["mistakes_made"]
    response = shortener_agent.getShortenerAgent().invoke(
        {
            "mistakes_made": mistakes_made,
        }
    )
    return {"mistakes_made": response}


def call_empfehler(state):
    student_sentence_english = state["student_sentence_english"]
    examiner_sentence_english = state["examiner_sentence_english"]
    mistakes_made = state["mistakes_made"]
    response = empfehler_agent.getEmpfehlerAgent().invoke(
        {
            "student_sentence_english": student_sentence_english,
            "mistakes_made": mistakes_made,
            "examiner_sentence_english": examiner_sentence_english
        }
    )
    return {"mistakes_made": response.content}


def call_tool(state):
    messages = state["messages"]
    # We know the last message involves at least one tool call
    last_message = messages[-1]

    # We loop through all tool calls and append the message to our message log
    for tool_call in last_message.additional_kwargs["tool_calls"]:
        action = ToolInvocation(
            tool=tool_call["function"]["name"],
            tool_input=json.loads(tool_call["function"]["arguments"]),
            id=tool_call["id"],
        )

        # We call the tool_executor and get back a response
        response = tool_executor.invoke(action)
        # We use the response to create a FunctionMessage
        function_message = ToolMessage(
            content=str(response), name=action.tool, tool_call_id=tool_call["id"]
        )

        # Add the function message to the list
        messages.append(function_message)
    return {"messages": messages}


def should_continue(state):
    last_message = state["messages"][-1]
    # If there are no tool calls, then we finish
    if "tool_calls" not in last_message.additional_kwargs:
        return "end"
    # Otherwise, we continue
    else:
        return "continue"

In [ ]:
from langgraph.graph import StateGraph, END

# Initialize a new graph 
graph = StateGraph(AgentState)

graph.add_node("grammar_checker", call_grammar_checker)
graph.add_node("bewerter_agent", call_bewerter)
graph.add_node("shortener_agent", call_shortener)
graph.add_node("empfehler_agent", call_empfehler)
graph.add_node("examiner", call_examiner)
graph.add_node("action", call_tool)

graph.set_entry_point("grammar_checker")

graph.add_conditional_edges(
    "grammar_checker",
    lambda x: x["grammar_correct"],
    {
        "yes": "examiner",
        "no": "bewerter_agent",
    },
)

graph.add_conditional_edges(
    "examiner",
    should_continue,
    {
        "continue": "action",
        "end": END,
    },
)

graph.add_edge("action", END)
graph.add_edge("bewerter_agent", "shortener_agent")
graph.add_edge("shortener_agent", "empfehler_agent")
graph.add_edge("empfehler_agent", "examiner")

# We compile the entire workflow as a runnable
app_agent = graph.compile()

In [ ]:
from langchain_core.messages import HumanMessage

message = "I have been reading a car for two hours."

inputs = {
    "examiner_sentence_german": "Ich lese seit zwei Stunden ein Buch.",
    "student_sentence_english": message,
    #"student_sentence_english": "I have been reading a book for two hours.",
    "examiner_requested_tense": "Present Perfect Continuous",
    "examiner_sentence_english": "I have been reading a book for two hours.",
    "examiner_topics": get_knowledge_comma_separated(),
    "messages": [HumanMessage(content=message)]
}

for output in app_agent.with_config({"run_name": "Examiner"}).stream(inputs):
    # stream() yields dictionaries with output keyed by node name
    for key, value in output.items():
        print(f"Output from node '{key}':")
        print("---")
        print(value)
    print("\n---\n")
    